In [1]:
!pip install emoji

In [2]:
import numpy as np
import pandas as pd

/Users/mohit/.pyenv/versions/3.8.1/lib/python3.8/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [3]:
train = pd.read_csv('dataset/train_emoji.csv',header=None)
test = pd.read_csv('dataset/test_emoji.csv',header=None)

In [4]:
train.head()

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [5]:
import emoji as emoji

In [6]:
#emoji.EMOJI_ALIAS_UNICODE

In [7]:
emoji_dictionary = {"0": "\u2764\uFE0F",    # :heart: prints a black instead of red heart depending on the font
                    "1": ":baseball:",
                    "2": ":beaming_face_with_smiling_eyes:",
                    "3": ":downcast_face_with_sweat:",
                    "4": ":fork_and_knife:",
                   }

In [8]:
emoji.emojize(":fork_and_knife:")

'🍴'

In [9]:
for e in emoji_dictionary.values():
    print(emoji.emojize(e))

❤️
⚾
😁
😓
🍴


In [10]:
data = train.values
for i in range(10):
    print(data[i][0],emoji.emojize(emoji_dictionary[str(data[i][1])]))

never talk to me again 😓
I am proud of your achievements 😁
It is the worst day in my life 😓
Miss you so much ❤️
food is life 🍴
I love you mum ❤️
Stop saying bullshit 😓
congratulations on your acceptance 😁
The assignment is too long  😓
I want to go play ⚾


In [11]:
from keras.utils import to_categorical

Using TensorFlow backend.


In [20]:
XT = train[0]
Xt = test[0]

YT = to_categorical(train[1])
Yt = to_categorical(test[1])


print(XT.shape)
print(Xt.shape)
print(YT.shape)
print(Yt.shape)

(132,)
(56,)
(132, 5)
(56, 5)


In [22]:
embeddings = {}
with open('glove.6B.50d.txt',encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coeffs = np.asarray(values[1:],dtype='float32')
        
        #print(word)
        #print(coeffs)
        embeddings[word] = coeffs

In [23]:
def getOutputEmbeddings(X):
    
    embedding_matrix_output = np.zeros((X.shape[0],10,50))
    for ix in range(X.shape[0]):
        X[ix] = X[ix].split()
        for jx in range(len(X[ix])):
            embedding_matrix_output[ix][jx] = embeddings[X[ix][jx].lower()]
            
    return embedding_matrix_output

In [24]:
XT[1]

'I am proud of your achievements'

In [25]:
emb_XT = getOutputEmbeddings(XT)
emb_Xt = getOutputEmbeddings(Xt)

<ipython-input-23-73145de5bfac>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[ix] = X[ix].split()


In [26]:
print(emb_XT.shape)
print(emb_Xt.shape)


(132, 10, 50)
(56, 10, 50)


In [27]:
from keras.layers import LSTM, Dropout, Dense, Activation
from keras.models import Sequential

In [28]:
model = Sequential()
model.add(LSTM(64,input_shape=(10,50),return_sequences=True))
model.add(Dropout(0.4))
model.add(LSTM(64,input_shape=(10,50)))
model.add(Dropout(0.3))
model.add(Dense(5))
model.add(Activation('softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 10, 64)            29440     
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 64)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 325       
_________________________________________________________________
activation_1 (Activation)    (None, 5)                 0         
Total params: 62,789
Trainable params: 62,789
Non-trainable params: 0
__________________________________________________

In [29]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])
model.fit(emb_XT,YT,batch_size=32,epochs=40,shuffle=True,validation_split=0.1)

Train on 118 samples, validate on 14 samples
Epoch 1/40
118/118 [==============================] - 1s 10ms/step - loss: 1.5722 - acc: 0.3136 - val_loss: 1.6226 - val_acc: 0.0714
Epoch 2/40
118/118 [==============================] - 0s 753us/step - loss: 1.5167 - acc: 0.3559 - val_loss: 1.6483 - val_acc: 0.0714
Epoch 3/40
118/118 [==============================] - 0s 756us/step - loss: 1.4538 - acc: 0.3729 - val_loss: 1.6483 - val_acc: 0.2143
Epoch 4/40
118/118 [==============================] - 0s 802us/step - loss: 1.4069 - acc: 0.4068 - val_loss: 1.6262 - val_acc: 0.2143
Epoch 5/40
118/118 [==============================] - 0s 915us/step - loss: 1.3660 - acc: 0.4407 - val_loss: 1.5511 - val_acc: 0.2143
Epoch 6/40
118/118 [==============================] - 0s 770us/step - loss: 1.2681 - acc: 0.5000 - val_loss: 1.4190 - val_acc: 0.3571
Epoch 7/40
118/118 [==============================] - 0s 679us/step - loss: 1.1741 - acc: 0.5763 - val_loss: 1.4435 - val_acc: 0.3571
Epoch 8/40
118/118

In [31]:
model.evaluate(emb_Xt,Yt)

56/56 [==============================] - 0s 331us/step


[1.782335707119533, 0.5892857313156128]

In [32]:
pred = model.predict_classes(emb_Xt)

In [33]:
for i in range(30):
    print(' '.join(Xt[i]))
    print(emoji.emojize(emoji_dictionary[str(np.argmax(Yt[i]))]))
    print(emoji.emojize(emoji_dictionary[str(pred[i])]))

I want to eat
🍴
🍴
he did not answer
😓
😓
he got a raise
😁
😓
she got me a present
❤️
😁
ha ha ha it was so funny
😁
😁
he is a good friend
❤️
😁
I am upset
❤️
😓
We had such a lovely dinner tonight
❤️
😁
where is the food
🍴
🍴
Stop making this joke ha ha ha
😁
😓
where is the ball
⚾
⚾
work is hard
😓
😓
This girl is messing with me
😓
❤️
are you serious ha ha
😁
😓
Let us go play baseball
⚾
⚾
This stupid grader is not working
😓
😓
work is horrible
😓
😓
Congratulation for having a baby
😁
😁
stop messing around
😓
⚾
any suggestions for dinner
🍴
😁
I love taking breaks
❤️
❤️
you brighten my day
😁
❤️
I boiled rice
🍴
🍴
she is a bully
😓
❤️
Why are you feeling bad
😓
😓
I am upset
😓
😓
I worked during my birthday
😓
😁
My grandmother is the love of my life
❤️
❤️
enjoy your break
😁
⚾
valentine day is near
❤️
😁


In [34]:
with open("model.json", "w") as file:
    file.write(model.to_json())
model.save_weights("model.h5")

In [35]:
from keras.models import model_from_json

In [36]:
with open("model.json", "r") as file:
    model = model_from_json(file.read())
model.load_weights("model.h5")

In [37]:
test_str = "Hello how are you"
X = pd.Series([test_str])

In [38]:
emb_X = getOutputEmbeddings(X)

In [39]:
p = model.predict_classes(emb_X)

In [40]:
print(' '.join(X[0]))
print(emoji.emojize(emoji_dictionary[str(p[0])]))

Hello how are you
😁
